# Deploy Cognito

Set up user authentication with AWS Cognito.

## Prerequisites

- ✅ Run `01-deploy-athena.ipynb` first

## What This Notebook Does

1. Creates Cognito User Pool
2. Creates App Client with OAuth scopes
3. Creates test users
4. Saves configuration to SSM

## Next Notebook

- **03-deploy-mcp-server.ipynb**

In [ ]:
# AWS Initialization - Load credentials and create session
from utils.notebook_init import init_aws
from pathlib import Path

# This will:
# 1. Load credentials from .env file (if it exists)
# 2. Create and validate AWS session (env vars take precedence over SSO)
# 3. Return session, region, and account_id for use in this notebook
session, region, account_id = init_aws()

# Initialize AWS clients
cognito_client = session.client('cognito-idp', region_name=region)
ssm_client = session.client('ssm', region_name=region)

print(f"✅ Ready to proceed with AWS operations")
print(f"   Account ID: {account_id}")
print(f"   Region: {region}")

## Step 1: Run Cognito Setup

In [ ]:
import subprocess

result = subprocess.run(
    ['python', 'setup_cognito.py', '--region', region],
    cwd='deployment/cognito-setup',
    capture_output=True,
    text=True
)

print(result.stdout)
if result.returncode != 0:
    print('❌ Error:', result.stderr)
else:
    print('\n✅ Cognito setup complete!')
    print('\n📋 Configuration automatically saved to SSM Parameter Store')

## Step 2: Verify Cognito Configuration in SSM

The setup_cognito.py script automatically saves all configuration to SSM Parameter Store.
Run this cell to verify the parameters were saved correctly.

In [ ]:
# Verify Cognito Configuration in SSM Parameter Store

print("Verifying Cognito parameters in SSM Parameter Store...\n")

# List of parameters to check
parameters_to_check = [
    '/app/lakehouse-agent/cognito-user-pool-id',
    '/app/lakehouse-agent/cognito-app-client-id',
    '/app/lakehouse-agent/cognito-domain',
    '/app/lakehouse-agent/cognito-resource-server-id',
]

# Check each parameter
all_found = True
for param_name in parameters_to_check:
    try:
        response = ssm_client.get_parameter(Name=param_name)
        value = response['Parameter']['Value']
        # Mask sensitive values
        display_value = value[:30] + '...' if len(value) > 30 else value
        print(f'✅ {param_name}')
        print(f'   Value: {display_value}')
    except ssm_client.exceptions.ParameterNotFound:
        print(f'❌ {param_name} - NOT FOUND')
        all_found = False
    except Exception as e:
        print(f'⚠️  {param_name} - ERROR: {e}')
        all_found = False

# Check secure parameters (without displaying values)
secure_params = [
    '/app/lakehouse-agent/cognito-app-client-secret',
]

for param_name in secure_params:
    try:
        response = ssm_client.get_parameter(Name=param_name, WithDecryption=False)
        print(f'✅ {param_name} (SecureString)')
        print(f'   Value: ***MASKED***')
    except ssm_client.exceptions.ParameterNotFound:
        print(f'❌ {param_name} - NOT FOUND')
        all_found = False
    except Exception as e:
        print(f'⚠️  {param_name} - ERROR: {e}')

if all_found:
    print('\n✅ All Cognito parameters verified in SSM Parameter Store!')
else:
    print('\n⚠️  Some parameters are missing. Re-run the setup_cognito.py script.')

## Summary

✅ **Cognito Deployment Complete!**

**Test Users Created:**
- user001@example.com / TempPass123!
- user002@example.com / TempPass123!
- adjuster001@example.com / TempPass123!

**Next Steps:**
Run **04-deploy-mcp-server.ipynb**